The data obtained on this csv is coming form this query obtaining the results from the CDW
This notebook is to aggregate it and organize it in a way that can be use later on the analysis


```
select
   wsf.date,
   wpr.websession_landing_url_raw as URL,
   wpd.page_no_query_string as URLClean,
   tsd.traffic_source as Source,
   count(distinct wsf.websession_id) as Sessions,
   sum(wsf.total_pages_viewed) as Pageviews,
   sum(wsf.return_visitor) as Return
INTO platea_sessions
from cdm.warehouse.web_session_fct AS wsf
   inner join cdm.warehouse.websession_landing_webpage_raw_dim wpr using (websession_landing_webpage_raw_dk)
   inner join cdm.warehouse.webpage_dim wpd USING (webpage_dk)
   inner join cdm.warehouse.traffic_source_dim AS tsd USING (traffic_source_dk)
WHERE   wsf.tenantid = 'src_1qnv2ee86cv3SDVqJk0YUhK40mN'

AND (wpr.websession_landing_url_raw LIKE 'https://www.plateapr.com%' or wpr.websession_landing_url_raw LIKE 'https://mapa.plateapr.com%')
 AND wsf.is_botsession_ind != 1
GROUP BY
        1,2,3,4
ORDER BY
        1
 ```

In [15]:
import pandas as pd

In [16]:
data = pd.read_csv("./data/platea_sessions.csv", names=['date', 'url', 'url_clean', 'source', 'sessions', 'views', 'return_visitors'] )

In [17]:
data.head()

,date,url,url_clean,source,sessions,views,return_visitors
0,2022-02-06,https://www.plateapr.com/lugares-donde-para-vi...,www.plateapr.com/lugares-donde-para-visitar-cu...,ORGANIC,1,1,0
1,2022-02-06,https://www.plateapr.com/trails-en-puerto-rico...,www.plateapr.com/trails-en-puerto-rico-para-ha...,DIRECT,1,1,0
2,2022-02-06,https://www.plateapr.com/sign-up/?utm_source=i...,www.plateapr.com/sign-up/,PAID,1,1,0
3,2022-02-06,https://www.plateapr.com/sign-up/?utm_source=i...,www.plateapr.com/sign-up/,PAID,1,1,1
4,2022-02-06,https://www.plateapr.com/restaurantes-para-dis...,www.plateapr.com/restaurantes-para-disfrutar-e...,PAID,1,1,1


In [18]:
data['url_clean'] = ['https://' + url if url.startswith('www.plateapr.com') else url for url in data['url_clean'] ]

In [19]:
to_save = data.groupby(['url_clean']).sum()
to_save.to_csv('./data/sessions_sum.csv')

In [6]:
data['url_clean'].value_counts()

https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/        10581
mapa.plateapr.com/places                                                      4631
https://www.plateapr.com/sign-up/                                             3000
https://www.plateapr.com/app/callback/                                        2779
https://www.plateapr.com/                                                     2419
                                                                             ...  
https://www.plateapr.com/search/restaurantes con vista al mar en san juan        1
https://www.plateapr.com/search/ocean view                                       1
https://www.plateapr.com/tag/hiking/                                             1
https://www.plateapr.com/search/cervesa                                          1
https://www.plateapr.com/search/san sebastián                                    1
Name: url_clean, Length: 1244, dtype: int64

In [7]:
to_save = data.groupby(['url_clean']).mean()
to_save.to_csv('./data/sessions_mean.csv')

In [20]:
to_save.describe()

,sessions,views,return_visitors
count,1244.000000,1244.000000,1244.000000
mean,407.227492,859.241961,114.267685
std,2072.341237,6517.641366,676.658420
min,1.000000,1.000000,0.000000
25%,1.000000,2.000000,1.000000
50%,4.000000,8.000000,2.000000
75%,35.250000,54.250000,17.000000
max,50215.000000,196765.000000,17534.000000


In [8]:
data['url_clean'].value_counts()

https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/        10581
mapa.plateapr.com/places                                                      4631
https://www.plateapr.com/sign-up/                                             3000
https://www.plateapr.com/app/callback/                                        2779
https://www.plateapr.com/                                                     2419
                                                                             ...  
https://www.plateapr.com/search/restaurantes con vista al mar en san juan        1
https://www.plateapr.com/search/ocean view                                       1
https://www.plateapr.com/tag/hiking/                                             1
https://www.plateapr.com/search/cervesa                                          1
https://www.plateapr.com/search/san sebastián                                    1
Name: url_clean, Length: 1244, dtype: int64

In [9]:
data.groupby('url_clean').agg({'sessions': 'sum', 'views':'sum', 'return_visitors':'sum'}).reset_index().sort_values(by=['sessions'], ascending=False)[0:30]

,url_clean,sessions,views,return_visitors
0,https://www.plateapr.com/,50215,196765,17534
430,https://www.plateapr.com/que-hacer-este-fin-de...,35055,87706,10516
984,mapa.plateapr.com/places,16199,20908,8477
311,https://www.plateapr.com/en/travel/puerto-rico...,13560,18813,1789
151,https://www.plateapr.com/en/food-drinks/best-b...,11263,13314,2221
287,https://www.plateapr.com/en/things-to-do/best-...,9931,11538,2370
154,https://www.plateapr.com/en/food-drinks/san-ju...,9101,11116,1617
445,https://www.plateapr.com/restaurantes-con-vist...,8899,14987,2276
129,https://www.plateapr.com/en/,7884,38196,2223
965,https://www.plateapr.com/unete-a-platea/,7411,31805,2458


#Original query was done 10/22/2022


To get the reminding data until 11/12/2022 used this query 
```
select
   wsf.date,
   wpr.websession_landing_url_raw as URL,
   wpd.page_no_query_string as URLClean,
   tsd.traffic_source as Source,
   count(distinct wsf.websession_id) as Sessions,
   sum(wsf.total_pages_viewed) as Pageviews,
   sum(wsf.return_visitor) as Return
INTO platea_sessions_2
from cdm.warehouse.web_session_fct AS wsf
   inner join cdm.warehouse.websession_landing_webpage_raw_dim wpr using (websession_landing_webpage_raw_dk)
   inner join cdm.warehouse.webpage_dim wpd USING (webpage_dk)
   inner join cdm.warehouse.traffic_source_dim AS tsd USING (traffic_source_dk)
WHERE   wsf.tenantid = 'src_1qnv2ee86cv3SDVqJk0YUhK40mN'
   AND wsf.date > '10/22/2022'

AND (wpr.websession_landing_url_raw LIKE 'https://www.plateapr.com%' or wpr.websession_landing_url_raw LIKE 'https://mapa.plateapr.com%')
 AND wsf.is_botsession_ind != 1
GROUP BY
        1,2,3,4
ORDER BY
        1
```

In [10]:
data2 = pd.read_csv("./data/platea_sessions_2.csv", names=['date', 'url', 'url_clean', 'source', 'sessions', 'views', 'return_visitors'] )

In [11]:
data2['url_clean'] = ['https://' + url if url.startswith('www.plateapr.com') else url for url in data2['url_clean'] ]


In [24]:
['https://' + url if url.startswith('www.plateapr.com') else url for url in data2['url_clean'] ]

['https://www.plateapr.com/en/food-drinks/san-juan-restaurants/',
 'https://www.plateapr.com/viking-arena-adventure-park-puerto-rico/',
 'https://www.plateapr.com/restaurantes-con-vista-al-mar/',
 'https://www.plateapr.com/restaurantes-veganos-en-puerto-rico/',
 'https://www.plateapr.com/en/things-to-do/best-nightlife-in-san-juan/',
 'https://www.plateapr.com/3-barras-para-ir-de-tapas-en-noviembre/',
 'https://www.plateapr.com/cervecera-de-puerto-rico-edicion-limitada-first-pick-arena-medalla/',
 'https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/',
 'https://www.plateapr.com/food-drinks/',
 'https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/',
 'https://www.plateapr.com/restaurantes-italianos-autenticos-en-san-juan/',
 'https://www.plateapr.com/en/food-drinks/family-friendly-restaurants/',
 'https://www.plateapr.com/mas-de-30-barras-se-unen-a-puerto-rico-cocktail-week-para-servir-cocteles-unicos-y-a-costos-reducidos/',
 'https://www.plateapr.com/q

In [12]:
import datetime
to_save = pd.concat([data,data2]).groupby(['url_clean']).sum()
to_save.to_csv(f'./data/sessions_{str(datetime.datetime.now())}.csv')



In [13]:
to_save

,sessions,views,return_visitors
url_clean,,,
https://www.plateapr.com/,52150,204776,18267
https://www.plateapr.com/ things to do best hikes el yunque,1,5,0
https://www.plateapr.com/.well-known/acme-challenge,1,7,1
https://www.plateapr.com/.well-known/change-password,1,9,0
https://www.plateapr.com/12-lugares-para-celebrar-el-dia-de-las-madres/,1538,2812,419
...,...,...,...
mapa.plateapr.com/trips/72670a2c-9bb4-45b4-bd7f-ff2c03d7ec9a,2,3,1
mapa.plateapr.com/trips/af9c1c07-8262-4624-a0b1-33d69a589242,1,1,1
mapa.plateapr.com/trips/f31fa524-b08a-4918-91f3-e353c483a32b,55,55,0


In [14]:
to_save.describe()

,sessions,views,return_visitors
count,1283.000000,1283.000000,1283.000000
mean,420.710055,880.949337,118.001559
std,2160.346852,6707.626254,710.158690
min,1.000000,1.000000,0.000000
25%,1.000000,2.000000,1.000000
50%,4.000000,8.000000,2.000000
75%,39.500000,57.500000,19.500000
max,52150.000000,204776.000000,18267.000000


In [23]:
to_save.sort_values(by=['sessions'], ascending=False).head(50)

,sessions,views,return_visitors
url_clean,,,
https://www.plateapr.com/,50215,196765,17534
https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/,35055,87706,10516
mapa.plateapr.com/places,16199,20908,8477
https://www.plateapr.com/en/travel/puerto-rico-declaration-form/,13560,18813,1789
https://www.plateapr.com/en/food-drinks/best-brunch-puerto-rico/,11263,13314,2221
https://www.plateapr.com/en/things-to-do/best-hikes-in-el-yunque-national-forest-in-puerto-rico/,9931,11538,2370
https://www.plateapr.com/en/food-drinks/san-juan-restaurants/,9101,11116,1617
https://www.plateapr.com/restaurantes-con-vista-a-las-montanas/,8899,14987,2276
https://www.plateapr.com/en/,7884,38196,2223
